In [1]:
# Importando os pacotes necessários
import py_dss_interface
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import math

In [2]:
# Criando o objeto "dss" e arquivo a ser compilado
dss_file = "C:\Projeto_Final\8500-Node\Master.dss"
dss = py_dss_interface.DSSDLL()

# Compilando arquivo com dados do sistema
dss.text("Clear")
dss.text("compile [{}]".format(dss_file))
# Inserindo um medidor de Energia no Alimentador que sai da subestação principal
alimentador = 'Line.ln5815900-1'
dss.text(f"New Energymeter.m1 {alimentador} 1")
# Inserindo um monitor das grandezas elétricas no Alimentador que sai da subestação principal
dss.text(f"New Monitor.m1_power {alimentador} terminal=1 mode=1 ppolar=False")
dss.text(f"New Monitor.m1_voltage {alimentador} terminal=1 mode=0 ppolar=False")
# Definindo itens de controle e iterações do software
dss.text("Set Maxiterations=20")
dss.text("Set maxcontrolit=100")

#Resolvendo Fluxo de Potência Estático
dss.solution_solve()

OpenDSS Started successfully! 
OpenDSS Version 9.4.0.1 (64-bit build); License Status: Open 




0

In [3]:
# Resultados de Potência Fornecida e Perdas no Sistema
P_MW = round(dss.circuit_total_power()[0]*-1/10**3,3)
Q_MVAr = round(dss.circuit_total_power()[1]*-1/10**3,3)
Total_Losses_MW = round(dss.circuit_losses()[0]/10**6,3)
Total_Losses_MVAR = round(dss.circuit_losses()[1]/10**6,3)
print(f"A potência ativa fornecida ao sistema é: {P_MW} MW")
print(f"A potência reativa fornecida ao sistema é: {Q_MVAr} MVAr")
print(f"O total de perdas ativas no sistema é {Total_Losses_MW} MW")
print(f"O total de perdas reativas no sistema é {Total_Losses_MVAR} MVAr")

A potência ativa fornecida ao sistema é: 11.984 MW
A potência reativa fornecida ao sistema é: 1.385 MVAr
O total de perdas ativas no sistema é 1.21 MW
O total de perdas reativas no sistema é 2.768 MVAr


In [4]:
# Construindo Lista de Cargas
loads_count = dss.loads_count()
loads_nome = list()
loads_kVA = list()
loads_kW = list()
loads_kVAr = list()


dss.loads_first()
for load in list(range(0,loads_count)):
    loads_nome.append(dss.loads_read_name()) 
    loads_kVA.append(dss.loads_read_kva())
    loads_kW.append(dss.loads_read_kw())
    loads_kVAr.append(dss.loads_read_kvar())
    dss.loads_next()

load_total_MVA = round((np.sum(loads_kVA)/10**3),2)
load_total_MW = round((np.sum(loads_kW)/10**3),2)
load_total_MVAr = round((np.sum(loads_kVAr)/10**3),2)

print(f"O sistema possui {loads_count} cargas, que resultam em uma potência instalada de: {load_total_MVA} MVA, sendo {load_total_MW} MW de potência ativa e\n{load_total_MVAr} MVAr de reativa")

O sistema possui 1177 cargas, que resultam em uma potência instalada de: 11.11 MVA, sendo 10.77 MW de potência ativa e
2.7 MVAr de reativa


In [46]:
# Informações do Sistema
info_sistema = losses.groupby('Tipo')['Tipo'].count()
info_sistema

Tipo
CapControl        9
Capacitor        10
EnergyMeter       1
Line           3703
Load           1177
Monitor           2
Reactor           1
RegControl       12
Transformer    1190
Vsource           1
Name: Tipo, dtype: int64

In [37]:
# Criando listas com nomes dos elementos e suas perdas em kW e kVAr
elementos_losses = dss.circuit_all_element_losses()
elementos = dss.circuit_all_element_names()
losses_kW = elementos_losses[::2]
losses_kVAr = elementos_losses[1::2]

# Criando dataframe com as perdas por equipamento
losses = pd.DataFrame(elementos,columns=['elementos'])
losses['losses_kW'] = losses_kW
losses['losses_kVAr'] = losses_kVAr
losses['Tipo'] = losses['elementos'].str.split(pat=".",expand=True)[0]
losses = losses.set_index('elementos')

# Perdas nas Linhas
line_losses = losses.query('Tipo=="Line"')

# LT com maior perda de potência ativa
max_line_losses = line_losses[(line_losses.losses_kW==line_losses.max()[0])]
max_ll_kw = max_line_losses['losses_kW'][0]
max_ll_kvar = max_line_losses['losses_kVAr'][0]
max_ll_name = max_line_losses.index[0].upper()

In [ ]:
# Ativando a linha com maior perda técnica de potência ativa
dss.circuit_set_active_element(max_ll_name)
dss.cktelement_name()

# Lendo número de fases da linha
line_phases = dss.cktelement_num_phases()

# Lendo as barras que a linha está conectada
line_bus1 = dss.cktelement_read_bus_names()[0].upper()
line_bus2 = dss.cktelement_read_bus_names()[1].upper()

# Lendo a tensão de base da barra 1 da linha
dss.circuit_set_active_bus(line_bus1)
bus1_kvbase = dss.bus_kv_base()*math.sqrt(3)

# Potência Ativa que entra e sai da linha
pkw_max_ll_bus1 = sum(dss.cktelement_powers()[0:6:2])
pkw_max_ll_bus2 = sum(dss.cktelement_powers()[6:11:2])

# Potência Ativa que entra e sai da linha
pkvar_max_ll_bus1 = sum(dss.cktelement_powers()[1:6:2])
pvar_max_ll_bus2 = sum(dss.cktelement_powers()[7:12:2])

In [ ]:
# Perdas nos Equipamentos e informações
print(f"A linha com mais perdas técnicas é a {max_ll_name} e possui {line_phases} fases")
print(f"A LT está conectada às barras {line_bus1} e {line_bus2}")
print(f"A tensão de base das barras cujo a {max_ll_name} está conectada é: {bus1_kvbase} kV")
print(f"A potência ativa na entrada da {max_ll_name} é {round(pkw_max_ll_bus1, 3)} kW")
print(f"A potência ativa na saída da {max_ll_name} é {round(-1*pkw_max_ll_bus2,3)} kW")
print("\nAs Perdas Técnicas de em kW e kVAr são mostradas na tabela abaixo")
max_line_losses.head()

In [ ]:
# Criando Listas das Tensões nos nós em PU
vmag_list_pu = dss.circuit_all_bus_vmag_pu()
nodes_names = dss.circuit_all_node_names()

In [ ]:
# Lista de todas as barras do sistema
bus_list = dss.circuit_all_bus_names()

# Lista de barras trifásicas
bus_3ph_list = list()
bus_kvbase_dict = dict()

for bus in bus_list:
    dss.circuit_set_active_bus(bus)
    num_fases = len(dss.bus_nodes())
    kv_base = dss.bus_kv_base()

    if num_fases == 3 and kv_base > (11.0/math.sqrt(3)):
        bus_3ph_list.append(bus)
        bus_kvbase_dict[bus] = kv_base

In [ ]:
# Encontrando a barra trifásica com menor tensão em pu
vmin = 20000

for bus in bus_3ph_list:
    dss.circuit_set_active_bus(bus)
    v_min = min(dss.bus_pu_vmag_angle()[::2])
    if v_min < vmin:
        nome = dss.bus_name()
        base_kV = (dss.bus_kv_base()*math.sqrt(3))
        vmin = v_min

print(f"A barra com menor tensão trifásica é a {nome.upper()}")
print(f"A tensão base dessa barra é: {base_kV} kV")
print(f"A menor tensão VLN em PU de uma barra trifásica é: {round(vmin,3)} pu")

In [ ]:
# Pegando a Barra com menor tensão em PU
vmin = min(vmag_list_pu) #menor valor da lista de tensões
vmin_index = vmag_list_pu.index(vmin) #índice da menor tensão na lista
node_vmin = nodes_names[vmin_index] #nome do nó segundo o índice
bus_vmin = node_vmin.split(".")[0].upper() #nome da barra com menor tensão em PU, transformando em maiúsculo
dss.circuit_set_active_bus(bus_vmin)
bus_vmin_kvbase = dss.bus_kv_base()*math.sqrt(3)
print(f"A barra com menor tensão, em PU, é a barra {bus_vmin}, que no fluxo de potência está com {round(vmin,3)} pu, equivalente à {round(vmin*bus_vmin_kvbase,3)} kV")
print(f"A tensão de base dessa barra é {bus_vmin_kvbase} kV")

In [ ]:
# Plotando o Circuito com marcações das barras em que a LT com maior perda técnica está conectada.
dss.text("AddBusMarker bus=M3032977 color=red size=2 code=15")
dss.text("AddBusMarker bus=M1166366 color=green size=2 code=15")
dss.text("Plot Circuit Power Max=5000 dots=n labels=n C1=Blue 1ph=3")